## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,tasiilaq,GL,65.6145,-37.6368,-2.00,59,74,11.32,broken clouds
1,1,husavik,IS,66.0449,-17.3389,22.91,65,100,18.28,overcast clouds
2,2,santa cruz de la palma,ES,28.6835,-17.7642,73.13,64,0,1.14,clear sky
3,3,nemuro,JP,43.3236,145.5750,19.94,78,9,7.02,clear sky
4,4,shimoda,JP,34.6667,138.9500,37.11,95,100,18.16,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
14,14,puerto ayora,EC,-0.7393,-90.3518,82.36,75,65,7.72,broken clouds
16,16,bathsheba,BB,13.2167,-59.5167,79.41,65,75,13.80,broken clouds
20,20,rikitea,PF,-23.1203,-134.9692,79.00,68,0,15.26,clear sky
21,21,pak chong,TH,14.7080,101.4161,75.81,59,85,3.22,light rain
30,30,bengkulu,ID,-3.8004,102.2655,76.26,62,100,3.18,overcast clouds
34,34,praya,CV,14.9215,-23.5087,77.54,47,0,13.80,clear sky
36,36,pimenta bueno,BR,-11.6725,-61.1936,79.20,81,98,2.86,overcast clouds
37,37,avarua,CK,-21.2078,-159.7750,82.45,74,20,12.66,few clouds
38,38,guanica,PR,17.9716,-66.9080,82.51,67,6,11.01,clear sky
45,45,hilo,US,19.7297,-155.0900,77.25,71,100,4.61,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                185
City                   185
Country                185
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy().dropna()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
14,puerto ayora,EC,82.36,-0.7393,-90.3518,
16,bathsheba,BB,79.41,13.2167,-59.5167,
20,rikitea,PF,79.00,-23.1203,-134.9692,
21,pak chong,TH,75.81,14.7080,101.4161,
30,bengkulu,ID,76.26,-3.8004,102.2655,
34,praya,CV,77.54,14.9215,-23.5087,
36,pimenta bueno,BR,79.20,-11.6725,-61.1936,
37,avarua,CK,82.45,-21.2078,-159.7750,
38,guanica,PR,82.51,17.9716,-66.9080,
45,hilo,US,77.25,19.7297,-155.0900,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params['location'] = f'{lat},{lng}'
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
 
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except Exception as e:
        print("hotel not found")
    

hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found
hotel not found


In [8]:
hotel_df["Hotel Name"]!= ""

14      True
16      True
20      True
21      True
30      True
       ...  
656     True
658     True
660     True
667     True
672    False
Name: Hotel Name, Length: 185, dtype: bool

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
14,puerto ayora,EC,82.36,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,bathsheba,BB,79.41,13.2167,-59.5167,Atlantis Hotel
20,rikitea,PF,79.00,-23.1203,-134.9692,People ThankYou
21,pak chong,TH,75.81,14.7080,101.4161,PakChong Phubade Hotel
30,bengkulu,ID,76.26,-3.8004,102.2655,Grage Hotel Bengkulu
34,praya,CV,77.54,14.9215,-23.5087,LT Aparthotel
36,pimenta bueno,BR,79.20,-11.6725,-61.1936,Cristalina Palace Hotel
37,avarua,CK,82.45,-21.2078,-159.7750,Paradise Inn
38,guanica,PR,82.51,17.9716,-66.9080,Copamarina Beach Resort & Spa
45,hilo,US,77.25,19.7297,-155.0900,Hilo Hawaiian Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

KeyError: 'Current Description'

In [ ]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig